In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os

In [7]:
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    # height
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])

def average_slope_intercept(image, lines):
    # contains coordinates of left and right lines for road
    left_fit = []
    right_fit = []
    for line in lines:
        # line is 2d array with 3 columns, we are reshaping it to 1d array with 4 elements
        x1, y1, x2, y2 = line.reshape(4)
        # fits polynomial to our x and y points and return a vector of coefficient which describes our x and y intersect
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        # if line we are iterating through has a negative slope value its the left line. Therefore append to left at tuple
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    try:
        # now we need a single why slope intercept
        left_fit_average = np.average(left_fit, axis = 0)
        right_fit_average = np.average(right_fit, axis = 0)
        left_line = make_coordinates(image, left_fit_average)
        right_line = make_coordinates(image, right_fit_average)
        return np.array([left_line, right_line])
    except:
        return None

def canny_process(image):
    # Convert image to gray scale
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    # plt.imshow(gray, cmap="gray")
    # plt.show()

    # Removes the strong features by bluring image
    # src − A Mat object representing the source (input image) for this operation.
    # kernel_size − A Size object representing the size of the kernel, specify the width and height of the kernel which should be positive and odd.
    # We also should specify the standard deviation in the X and Y directions, sigmaX and sigmaY respectively. If only sigmaX is specified, sigmaY is taken as equal to sigmaX. If both are given as zeros, they are calculated from the kernel size.
    blur = cv2.GaussianBlur(gray,(5, 5), 0)   # Kernel size is 5x5
    # plt.imshow(blur, cmap="gray")
    # plt.show()

    # Canny detects edges in an image 
    # low_threshold, high_threshold values to help assign how sensitive it finds edges
    canny = cv2.Canny(blur, 50, 150)
    # plt.imshow(canny, cmap="gray")
    # plt.show()
    return canny


def display_lines(image, lines):
    # create empty image with same dimensions as image passed in
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            # Draw each line to empty image
            # #1 Image
            # #2 First point of line
            # #3 Second point of line
            # #4 Color of line
            # #5 Line thickness
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

# This only keeps the region of the image defined by a polygon. The rest of the image is set to black.
def region_of_interest(image):
    # Gets height of image based in pixels
    height = image.shape[0]
    # Make Triangle based on image shape. Values can be found on plt.show() x and y axis value
    polygons = np.array([
        [(200,height),(1100,height),(550,250)]
    ])
    # Creates array of zeros with the same * size as image passed in
    mask = np.zeros_like(image)
    # Fill mask
    # With triangle
    # Color of triangle
    cv2.fillPoly(mask, polygons, 255)

    # Computing the bitwise & of both images. 
    # Takes the bitwise & of each homologou pixel in both arrays, ultimately masking the canny image to only show the region of interest traced by the polygonal contour of the mask
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image



# for img in os.listdir('Test_images/'):
#     # Lane image
#     image = cv2.imread('Test_images/' + img)

cap = cv2.VideoCapture("test2.mp4")
while(cap.isOpened()):
    _, image = cap.read()
    # modified image, grey scaled, blur, etc
    cannyImage = canny_process(image)
    # cropped area of interest where the road is most likely to be
    croppedImage =  region_of_interest(cannyImage)
    # detects line in image
    # #1 Our Image
    # #2 && #3 Similar to grid or 2d array (we need two values to decide the size of the been) kinda like a view port or section of scanning over image
    # it will check each section and see if there is interestion betwene lines to figure our the precise angle 
    # #4 Gradiant threshhold (threshold need to be atleast 100 to be seen as a line)
    # #5 placeholder array 
    # #6 any lines traced less 40 pixels are rejected
    # #7 maximum pixel gap of a line that is acceptable. if gap is 3px its still considered one line and not 2 seperate lines
    lines = cv2.HoughLinesP(croppedImage, 2, np.pi/180, 100, np.array([]), minLineLength = 40, maxLineGap = 5)
    average_lines = average_slope_intercept(image, lines)
    lineImage = display_lines(image, average_lines)

    # This takes the weighed sum between the 2 images
    # Image
    # Pixel intensity multiply by number
    # Image
    # Pixel intensity multiply by number (This has higher weight making the line more visible then the other image)
    # ScaLer value of 1
    joinImage = cv2.addWeighted(image, 0.8, lineImage, 1, 1 )

    cv2.imshow("result",joinImage)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
